# Install and load packages

In [1]:
!pip install -q ohsome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from ohsome import OhsomeClient
import folium

In [3]:
# mount the google drive (if operating from within colab)
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/master_thesis/predicting_poverty/preprocess_OSM

Mounted at /content/gdrive
/content/gdrive/MyDrive/master_thesis/predicting_poverty/preprocess_OSM


In [4]:
# load the OSM package
from OSM_utils.process import *

# Load the LSMS data

In [5]:
# define the column names of lat and lon
lat, lon = 'lat', 'lon'
LSMS_CSV_PATH = '../../Data/lsms/processed/labels_cluster_v1.csv'
lsms_df = pd.read_csv(LSMS_CSV_PATH)
lsms_df = lsms_df[['country', 'cluster_id', lat, lon]]
cluster_df = lsms_df.drop_duplicates().reset_index(drop = True).copy()
cluster_df

,country,cluster_id,lat,lon
0,eth,eth_010101088801601,14.358684,37.912338
1,eth,eth_010102088801403,14.288045,38.220532
2,eth,eth_010103010100106,14.112234,38.473810
3,eth,eth_010103088801804,13.859122,38.464229
4,eth,eth_010105088800204,14.092654,37.963131
...,...,...,...,...
2250,uga,uga_4140005,-0.907206,29.811566
2251,uga,uga_4150002,0.600000,30.650000
2252,uga,uga_4150006,0.587796,30.464436
2253,uga,uga_4150007,0.731301,30.642676


In [6]:
# first translate the dataset into a geopands dataframe containing the
# given coordinates as a point as well as the bounding box around it as a polygon
pt_geoms = gpd.points_from_xy(
    x=cluster_df[lon],
    y=cluster_df[lat],
    crs="EPSG:4326",
)

cluster_gpd = gpd.GeoDataFrame(cluster_df.copy(), geometry = pt_geoms)
# create buffer. First project into projection that calculates in meters

# create region of interest
cluster_gpd['roi'] = cluster_gpd['geometry'].to_crs("EPSG:3857").buffer(3360, cap_style = 3).to_crs("EPSG:4326")

# Get the number of the POIs and the distance to the closest POI
This is best done by country: For each country load the shape file and download all POIs within the country.
For each POI calculate the distance to the centroid of each ROI and count those that are within the ROI. Finally get the minimum distance to each POI.

As POIs include:<br>
1. Restraunt, Bar, Cafes, market place
2. School, Universities, libraries
3. Fuel station, bus station
4. pharmacy, health_post, hospital, doctors, clinic, drinking_water

Do not consider health_post, bus_station, doctors, drinking_water due to low occurence --> just not mapped objects

In [8]:
# shape file paths
path_dict = {'eth': '../../Data/geoboundaries/shapefiles/eth_shp/gadm41_ETH_0.shp',
             'mwi': '../../Data/geoboundaries/shapefiles/mwi_shp/gadm41_MWI_0.shp',
             'nga': '../../Data/geoboundaries/shapefiles/nga_shp/NGA_adm_0.shp',
             'uga': '../../Data/geoboundaries/shapefiles/uga_shp/gadm41_UGA_0.shp',
             'tza': '../../Data/geoboundaries/shapefiles/tza_shp/gadm41_TZA_0.shp'}

shp = {}
# load shapefile
for key in path_dict.keys():
  shp[key] = gpd.read_file(path_dict[key])

## Amenity Count

In [9]:
client = OhsomeClient(log=False)

#countries = np.unique(cluster_df['country'])

# amenity_features = []

#   #fltr = '(amenity in (marketplace,school))'

# create the polygon dictionary
bboxes = {}
for idx, cluster_info in cluster_gpd.iterrows():
  cluster_id = cluster_info.cluster_id
  bbox = list(cluster_info.roi.bounds)
  #bboxes =  f'{cluster_id}:{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}|'
  bboxes[cluster_id] = bbox

amenities = ['restaurant', 'bar', 'cafe', 'marketplace','school', 'university', 'library', 'fuel', 'pharmacy', 'hospital', 'clinic']
amenity_counts = {}

for amn in tqdm(amenities):
  print(f"Process amenity {amn}")
  fltr = f'amenity={amn}'

  # count elements in for each bpoly
  tm = "2023-03-01/2023-03-26/P1M"
  response = client.elements.count.groupByBoundary.post(bboxes = bboxes,
                                        time=tm,
                                        filter=fltr)
  amenity_count_df = response.as_dataframe().reset_index(drop = False)
  amenity_counts[amn] = amenity_count_df.value


  0%|          | 0/11 [00:00<?, ?it/s]

Process amenity restaurant
Process amenity bar
Process amenity cafe
Process amenity marketplace
Process amenity school
Process amenity university
Process amenity library
Process amenity fuel
Process amenity pharmacy
Process amenity hospital
Process amenity clinic


In [10]:
amenity_counts['cluster_id'] = cluster_gpd['cluster_id']
amenity_counts_df = pd.DataFrame(amenity_counts)
amenity_counts_df.to_csv('../../Data/OSM/osm_amenity_count.csv', index = False)

## Amenity Distance

In [11]:
client = OhsomeClient(log=False)

countries = np.unique(cluster_df['country'])

amenity_distances = []
for cntry in countries:
  print(f"country {cntry}")

  bpolys = shp[cntry]
  fltr = 'amenity in (restaurant, bar, cafe, marketplace, school, university, library, fuel, pharmacy, hospital, clinic)'
  amenities = download_OSM_data(bpolys, fltr, client, tags = True)

  # preprocess reponse, ensure all geometries valid, ensure only polygons, not multipolygons
  amenities = preprocess_response(amenities)

  # store the results of the query in a dictionary by amenity (easier to handle)
  amn_dict = {}
  for amn in amenities.amenity.value_counts().keys():
    amn_mask = amenities['amenity'] == amn
    amn_df = amenities.loc[amn_mask,:].copy()
    amn_df['geometry'] = amn_df['geometry'].to_crs('EPSG:3857')
    amn_dict[amn] = amn_df

  # subset the cluster dataframe to the current country
  cntry_gpd = cluster_gpd[cluster_gpd['country'] == cntry].copy().reset_index(drop = True)
  proj_points = cntry_gpd['geometry'].to_crs('EPSG:3857')

  # for each ROI in the cntry_gpd calculate the number of occurences for each amenity
  # and the distance to the closest amenity
  cluster_amenity_distances = []

  for point in tqdm(proj_points):
    amn_distances = {}
    for amn in amn_dict.keys():
      dist = np.array([i.distance(point) for i in amn_dict[amn]['geometry']])
      min_dist = float(min(dist))
      amn_distances[amn] = min_dist
    cluster_amenity_distances.append(amn_distances)
  country_amenity_distances = pd.json_normalize(cluster_amenity_distances)
  country_amenity_distances['cluster_id'] = cntry_gpd['cluster_id']
  amenity_distances.append(country_amenity_distances)

country eth
fixing 0 invalid geometries


  0%|          | 0/431 [00:00<?, ?it/s]

country mwi
fixing 0 invalid geometries


  0%|          | 0/204 [00:00<?, ?it/s]

country nga
fixing 0 invalid geometries


  0%|          | 0/480 [00:00<?, ?it/s]

country tza
fixing 0 invalid geometries


  0%|          | 0/826 [00:00<?, ?it/s]

country uga
fixing 0 invalid geometries


  0%|          | 0/314 [00:00<?, ?it/s]

In [12]:
# store results
amenity_distances_df = pd.concat(amenity_distances).reset_index(drop = True)
amenity_distances_df.to_csv("../../Data/OSM/osm_amenity_distances.csv", index = False)

# Road features
Extract information for each cluster on the length of roads as well as the road quality within each ROI.

## Length of the road network within each ROI

In [13]:
client = OhsomeClient(log=False)
fltr = "highway in (primary, seconday, tertiary, unclassified, residential, trunk, motorway)"
bboxes = {}
for idx, cluster_info in cluster_gpd.iterrows():
  cluster_id = cluster_info.cluster_id
  bbox = list(cluster_info.roi.bounds)
  #bboxes =  f'{cluster_id}:{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}|'
  bboxes[cluster_id] = bbox

tm = "2023-03-01/2023-03-26/P1M"
print("Downloading OSM Data...")
response = client.elements.length.groupByBoundary.post(bboxes = bboxes,
                                                       time=tm,
                                                       filter=fltr)
road_length = response.as_dataframe().reset_index(drop = False)

In [14]:
road_length = road_length.rename(columns = {"boundary":"cluster_id", 'value':'road_length'})
road_length_df = road_length[['cluster_id','road_length']]

## Distance to the nearest primary road and to the nearest paved road.

In [15]:
client = OhsomeClient(log=False)

countries = np.unique(cluster_df['country'])

road_distances = []
for cntry in countries:
  print(f"country {cntry}")

  bpolys = shp[cntry]
  fltr = "(highway in (trunk, motorway)) or (highway in (primary, secondary, tertiary, unclassified, residential) and (surface in (paved, asphalt, concrete)))"
  paved_roads = download_OSM_data(bpolys, fltr, client, tags = False)
  # preprocess reponse, ensure all geometries valid, ensure only polygons, not multipolygons
  paved_roads = preprocess_response(paved_roads).to_crs("EPSG:3857")
  road_length = paved_roads.geometry.length
  mask = road_length > 500
  paved_roads = paved_roads.loc[mask].reset_index(drop= True)

  fltr = "highway=primary"
  primary_roads = download_OSM_data(bpolys, fltr, client, tags = False)
  primary_roads = preprocess_response(primary_roads).to_crs("EPSG:3857")

  # subset the cluster dataframe to the current country
  cntry_gpd = cluster_gpd[cluster_gpd['country'] == cntry].copy().reset_index(drop = True)
  proj_points = cntry_gpd['geometry'].to_crs('EPSG:3857')

  # for each ROI in the cntry_gpd calculate the distance from its centroid to the nearest
  # paved and primary road
  cluster_road_distances = []

  for point in tqdm(proj_points):
    distances = {}
    dist_paved = np.array([i.distance(point) for i in paved_roads['geometry']])
    distances['paved'] = float(min(dist_paved))

    dist_primary = np.array([i.distance(point) for i in primary_roads['geometry']])
    distances['primary'] = float(min(dist_primary))

    cluster_road_distances.append(distances)

  country_road_distances = pd.json_normalize(cluster_road_distances)
  country_road_distances['cluster_id'] = cntry_gpd['cluster_id']
  road_distances.append(country_road_distances)

country eth
fixing 0 invalid geometries
fixing 0 invalid geometries


  0%|          | 0/431 [00:00<?, ?it/s]

country mwi
fixing 0 invalid geometries
fixing 0 invalid geometries


  0%|          | 0/204 [00:00<?, ?it/s]

country nga
fixing 0 invalid geometries
fixing 0 invalid geometries


  0%|          | 0/480 [00:00<?, ?it/s]

country tza
fixing 0 invalid geometries
fixing 0 invalid geometries


  0%|          | 0/826 [00:00<?, ?it/s]

country uga
fixing 0 invalid geometries
fixing 0 invalid geometries


  0%|          | 0/314 [00:00<?, ?it/s]

In [16]:
# merge the road feature data and store it
road_distances_df = pd.concat(road_distances).reset_index(drop = True)
road_features = pd.merge(road_length_df, road_distances_df, on = 'cluster_id')
road_features = road_features.rename(columns = {"paved":'distance_paved','primary':'distance_primary'})
road_features.to_csv("../../Data/OSM/osm_road_features.csv", index = False)

In [18]:
location = [14.325394413810697,37.87797724038243]
old_loc = [14.353816,37.890876]
m = folium.Map(location)
#folium.GeoJson(cluster_gpd.roi[0]).add_to(m)
#folium.GeoJson(cluster_gpd.geometry[0]).add_to(m)
folium.GeoJson(primary_roads.geometry).add_to(m)
#folium.GeoJson(pop_places_df.geometry).add_to(m)
m
#bboxes